# PCA + KMeans Clustering (Reproducible Pipeline)

This notebook clusters companies using a pre-built, **scaled feature matrix** (for modeling) and links the resulting cluster labels back to a more readable **original dataset** (for interpretation). It produces two outputs: (1) the original dataset with a `cluster` column, and (2) a cluster profile summary computed on original features.

In [ ]:
# B) Imports (consolidated)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import silhouette_score

In [ ]:
# C) Config / File Paths (centralized)
SCALED_PATH = "../data/processed/features_for_clustering_scaled.csv"
ORIGINAL_PATH = "../data/processed/cleaned_base.csv"

OUT_ORIGINAL_WITH_CLUSTERS = "../data/processed/original_with_clusters.csv"
OUT_CLUSTER_PROFILE = "../data/processed/cluster_summary_original_features.csv"

RANDOM_STATE = 42

In [ ]:
# D) Load Data (scaled features + original readable dataset)
df_scaled = pd.read_csv(SCALED_PATH)
df_original = pd.read_csv(ORIGINAL_PATH, low_memory=False)

print("df_scaled shape   :", df_scaled.shape)
print("df_original shape :", df_original.shape)

# Validate row alignment (must be same row order for label linking)
if len(df_scaled) != len(df_original):
    raise ValueError(
        f"Row mismatch! df_scaled has {len(df_scaled)} rows but df_original has {len(df_original)} rows.\n"
        "They MUST represent the same entities in the same order so cluster labels can be linked back safely.\n"
        "Fix upstream preprocessing to avoid dropping/shuffling rows, or re-export the aligned files."
    )

# Optional stronger check if a stable row id exists in both files
candidate_id_cols = ["row_id", "RowID", "id", "ID"]
id_col = next((c for c in candidate_id_cols if c in df_scaled.columns and c in df_original.columns), None)
if id_col is not None:
    if not (df_scaled[id_col].astype(str).values == df_original[id_col].astype(str).values).all():
        raise ValueError(
            f"Row order mismatch detected via column '{id_col}'.\n"
            "The two files must be aligned row-by-row before clustering labels can be joined back."
        )

## E) Dimensionality Reduction

We reduce dimensionality to make KMeans faster and more stable.
- If the feature matrix is **dense** (like a numeric CSV), we use **PCA** with `n_components=0.95` to keep 95% variance.
- If the feature matrix is **sparse** (common after one-hot encoding), PCA may require centering and can become expensive; in that case we use **TruncatedSVD**, which works directly with sparse matrices without densifying.

In [ ]:
# E2) Compute PCA (dense) or TruncatedSVD (sparse) -> df_pca
X = df_scaled

# Detect sparse matrices without forcing a SciPy dependency (SciPy is optional here)
is_sparse = False
try:
    from scipy import sparse  # optional; only used for sparse checks
    is_sparse = sparse.issparse(X)
except Exception:
    is_sparse = False

if is_sparse:
    # TruncatedSVD does not support n_components as a variance ratio directly.
    # Use a reasonable cap; increase if cumulative variance is too low for your use case.
    n_components = min(200, X.shape[1] - 1) if X.shape[1] > 1 else 1
    reducer = TruncatedSVD(n_components=n_components, random_state=RANDOM_STATE)
    X_reduced = reducer.fit_transform(X)
    explained = float(np.sum(reducer.explained_variance_ratio_))
    reducer_name = "TruncatedSVD"
else:
    reducer = PCA(n_components=0.95, random_state=RANDOM_STATE)
    X_reduced = reducer.fit_transform(X)
    explained = float(np.sum(reducer.explained_variance_ratio_))
    reducer_name = "PCA"

df_pca = pd.DataFrame(X_reduced, columns=[f"PC{i+1}" for i in range(X_reduced.shape[1])])

print(f"Reducer: {reducer_name}")
print("Original shape:", X.shape)
print("Reduced shape :", df_pca.shape)
print("Explained variance (sum):", explained)

df_pca.head()

## F) Choose K

We evaluate a small range of cluster counts ($k=2$ to $10$) using:
- **Silhouette score** (higher is better)
- **Inertia** (elbow method; lower is better, diminishing returns)

By default, we select `best_k` from the highest silhouette score (you can override if the elbow plot suggests a different tradeoff).

In [ ]:
# F2) K selection: silhouette + elbow
K_range = range(2, 11)
sil_scores = {}
inertias = {}

n_samples = len(df_pca)
for k in K_range:
    if k >= n_samples:
        break
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    labels = km.fit_predict(df_pca)
    sil = silhouette_score(df_pca, labels)
    sil_scores[k] = float(sil)
    inertias[k] = float(km.inertia_)
    print(f"k={k}, silhouette={sil:.4f}, inertia={km.inertia_:.2f}")

best_k = max(sil_scores, key=sil_scores.get)
print("\nBest k by silhouette:", best_k)
# best_k = 5  # Uncomment to override manually

# Plots (shown once)
plt.figure(figsize=(7, 4))
plt.plot(list(inertias.keys()), list(inertias.values()), marker="o")
plt.xlabel("k")
plt.ylabel("Inertia")
plt.title("Elbow Method (KMeans)")
plt.grid(True)
plt.show()

plt.figure(figsize=(7, 4))
plt.plot(list(sil_scores.keys()), list(sil_scores.values()), marker="o")
plt.xlabel("k")
plt.ylabel("Silhouette score")
plt.title("Silhouette Score vs k")
plt.grid(True)
plt.show()

In [ ]:
# G1) Train final KMeans
kmeans = KMeans(n_clusters=best_k, random_state=RANDOM_STATE, n_init=10)
cluster_labels = kmeans.fit_predict(df_pca)

cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
print("Cluster counts:")
display(cluster_counts)

In [ ]:
# G2) Visualize: PC1 vs PC2 colored by cluster
if df_pca.shape[1] < 2:
    raise ValueError("Need at least 2 components to plot PC1 vs PC2.")

plt.figure(figsize=(8, 6))
plt.scatter(df_pca["PC1"], df_pca["PC2"], c=cluster_labels, s=10)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Clusters (PC space)")
plt.grid(True)
plt.show()

In [ ]:
# H) Link clusters back to original data + save
df_original_with_clusters = df_original.copy()
df_original_with_clusters["cluster"] = cluster_labels

df_original_with_clusters.to_csv(OUT_ORIGINAL_WITH_CLUSTERS, index=False)
print("Saved original dataset with clusters to:", OUT_ORIGINAL_WITH_CLUSTERS)

df_original_with_clusters.head()

## I) Cluster Interpretation Summary

Interpretation should be done on **original features**, not on PC features. Below we build a simple `profile_display` table per cluster:
- cluster size
- means/medians for known numeric columns if present
- % coverage for known flag columns if present
- top value for known categorical columns if present

The code is written to be robust to missing columns (it will only summarize what exists).

In [ ]:
# I2) Build cluster profile summary on original features
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)

candidate_numeric = [
    "Employees Single Site",
    "Employees Total",
    "Revenue (USD)",
    "Year Found",
    "Market Value (USD)"
]
numeric_cols = [c for c in candidate_numeric if c in df_original_with_clusters.columns]

candidate_cats = [
    "Country",
    "Region",
    "Entity Type",
    "Ownership Type",
    "NAICS Description",
    "SIC Description",
    "Legal Status"
]
cat_cols = [c for c in candidate_cats if c in df_original_with_clusters.columns]

candidate_flags = ["has_website", "has_phone", "has_address", "has_city", "has_country"]
flag_cols = [c for c in candidate_flags if c in df_original_with_clusters.columns]

print("Summary columns used:")
print("- Numeric:", numeric_cols)
print("- Categorical:", cat_cols)
print("- Flags:", flag_cols)

def top_value(series: pd.Series):
    s = series.dropna()
    if len(s) == 0:
        return None
    return s.value_counts().index[0]

profile = df_original_with_clusters.groupby("cluster").agg(size=("cluster", "count"))

for col in numeric_cols:
    profile[f"{col}__mean"] = df_original_with_clusters.groupby("cluster")[col].mean()
    profile[f"{col}__median"] = df_original_with_clusters.groupby("cluster")[col].median()

for col in flag_cols:
    profile[f"{col}__pct"] = df_original_with_clusters.groupby("cluster")[col].mean() * 100

for col in cat_cols:
    profile[f"{col}__top"] = df_original_with_clusters.groupby("cluster")[col].apply(top_value)

needed_for_risk = ["has_website", "has_phone", "has_address"]
if all(c in df_original_with_clusters.columns for c in needed_for_risk):
    df_original_with_clusters["risk_score"] = (
        (1 - df_original_with_clusters["has_website"]) +
        (1 - df_original_with_clusters["has_phone"]) +
        (1 - df_original_with_clusters["has_address"])
    )
    profile["risk_score__mean"] = df_original_with_clusters.groupby("cluster")["risk_score"].mean()

profile = profile.sort_values("size", ascending=False)

profile_display = profile.copy()
for c in profile_display.columns:
    if "__mean" in c or "__median" in c:
        profile_display[c] = profile_display[c].round(2)
    if "__pct" in c:
        profile_display[c] = profile_display[c].round(1)
    if "risk_score" in c:
        profile_display[c] = profile_display[c].round(3)

profile_display.to_csv(OUT_CLUSTER_PROFILE, index=True)
print("Saved cluster summary to:", OUT_CLUSTER_PROFILE)

display(profile_display)

# Clean report view (subset)
important_cols = ["size"]
for col in ["Revenue (USD)__mean", "Revenue (USD)__median", "Employees Total__mean", "Employees Total__median"]:
    if col in profile_display.columns:
        important_cols.append(col)
for col in flag_cols:
    colname = f"{col}__pct"
    if colname in profile_display.columns:
        important_cols.append(colname)
for col in ["Country__top", "NAICS Description__top", "Ownership Type__top", "Entity Type__top"]:
    if col in profile_display.columns:
        important_cols.append(col)
if "risk_score__mean" in profile_display.columns:
    important_cols.append("risk_score__mean")

display(profile_display[important_cols])

In [ ]:
# J) (Optional) Inspect a single cluster
CLUSTER_TO_VIEW = 0  # change as needed

# Example: show 20 sample rows from the chosen cluster
df_original_with_clusters[df_original_with_clusters["cluster"] == CLUSTER_TO_VIEW].head(20)